In [1]:
from tqdm.notebook import tqdm
import clip
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

clip_model, clip_preprocess = clip.load("ViT-B/32", device)

cuda


In [2]:
from datasets import *
dataset_obj = Flowers102(0, 50)
train_loader, valid_loader = dataset_obj.get_train_loaders(transform_fn=clip_preprocess, shuffle=False)
test_loader = dataset_obj.get_test_loader(transform_fn=clip_preprocess)
classes = dataset_obj.classes

In [3]:
import copy

In [4]:
def get_clip_features(dataset):
    all_features = []
    all_labels = []

    global clip_model

    with torch.no_grad():
        for images, labels in tqdm(dataset):
            features = clip_model.encode_image(images.to(device))
            all_features.append(features)
            all_labels.append(labels)

    return torch.cat(all_features), torch.cat(all_labels)

train_features, train_labels = get_clip_features(train_loader)
valid_features, valid_labels = get_clip_features(valid_loader)
test_features, test_labels = get_clip_features(test_loader)

  0%|          | 0/132 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

In [5]:
def batch(iterable1,iterable2, n=1):
    l = len(iterable1)
    for ndx in range(0, l, n):
        yield (iterable1[ndx:min(ndx + n, l)], iterable2[ndx:min(ndx + n, l)])

out_train = batch(train_features,train_labels, 50)
out_test = batch(test_features,test_labels, 50)

In [19]:
import torch.nn as nn
import torch.optim as optim

class LogisticRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        outputs = self.linear(x)
        return outputs
    
model = LogisticRegression(512,len(classes))
model.logit_scale = nn.Parameter(torch.ones([], device=device))
criterion = nn.CrossEntropyLoss()
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-5)
num_epochs = 500

In [20]:
def num_correct_preds(outputs, labels):
    _, predicted = outputs.max(1)
    return predicted.eq(labels).sum().item()

In [21]:
def cosine_loss(output, target):
    loss = 1 - torch.cosine_similarity(output, target)
    return loss

def calc_loss(outputs, labels, loss_name="ce"):
    
    loss_labels = labels
    
    criterion = nn.CrossEntropyLoss()

    if loss_name == "ce":
        return criterion(outputs, loss_labels) 

    elif loss_name == "dot":
        outputs = outputs / outputs.norm(dim=-1, keepdim=True)
        return -(outputs * loss_labels).sum(-1).mean()

    elif loss_name == "cosine":
        print(outputs.shape, loss_labels.shape)
        loss = torch.mean(cosine_loss(outputs, loss_labels))
        return loss

    elif loss_name == "temperature_ce":
        image_features = outputs

        # normalized features
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)

        # cosine similarity as logits
        logit_scale = model.logit_scale.exp()
        logits_per_image = logit_scale * image_features @ text_features.t()
        return criterion(logits_per_image, labels)

In [22]:
import torch
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [23]:
import random

model.to(device)
model = model.to(torch.float16)

best_model = None
best_acc = -1

for epoch in tqdm(range(num_epochs+1)):
    
    model.train()
    running_loss = 0.0
    correct = 0.0
    total = 0

    if epoch%50 == 0:
        print("Shuffling")
        c = list(zip(train_features, train_labels))
        random.shuffle(c)
        train_features, train_labels = zip(*c)
        train_features = torch.stack(list(train_features))
        train_labels = torch.stack(list(train_labels))
        
    out_train = batch(train_features,train_labels, 50)
    for inputs, labels in out_train:
        
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = calc_loss(outputs, labels)

        loss.backward()

        optimizer.step()
        running_loss+=loss
        total += len(labels)
        correct += num_correct_preds(outputs, labels)
        
    if epoch%10 == 0:
        model.eval()
        out_test = batch(valid_features,valid_labels, 50)
        
        test_running_loss = 0.0
        test_correct = 0.0
        test_total = 0
        with torch.no_grad():
            for inputs, labels in tqdm(out_test):
        #         inputs = inputs / inputs.norm(dim=-1, keepdim=True)
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                test_running_loss += loss.item()
                test_total += labels.size(0)
                test_correct += num_correct_preds(outputs, labels)

        epoch_loss = test_running_loss / len(test_loader)
        writer.add_scalar("Loss/val", epoch_loss, epoch)
        epoch_accuracy = test_correct * 100 / test_total
        writer.add_scalar("Accuracy/val", epoch_loss, epoch)
        if epoch_accuracy > best_acc:
            best_model= copy.deepcopy(model)
            best_acc = epoch_accuracy
            print("Found best model")
#         print(
#             f"Testing: Epoch {epoch} || Loss: {epoch_loss:7.3f} || Accuracy: {epoch_accuracy:6.2f}%"
#         )

        
    epoch_loss = running_loss/len(train_loader)
    writer.add_scalar("Loss/train", epoch_loss, epoch)
    epoch_accuracy = correct*100/total
    writer.add_scalar("Accuracy/train", epoch_loss, epoch)
    print(
        f"Training: Epoch {epoch} || Loss: {epoch_loss:7.3f} || Accuracy: {epoch_accuracy:6.2f}%"
    )

    writer.flush()
writer.close()


  0%|          | 0/501 [00:00<?, ?it/s]

Shuffling


0it [00:00, ?it/s]

Found best model
Training: Epoch 0 || Loss:   4.121 || Accuracy:  29.13%
Training: Epoch 1 || Loss:   3.553 || Accuracy:  54.87%
Training: Epoch 2 || Loss:   2.674 || Accuracy:  61.26%
Training: Epoch 3 || Loss:   2.260 || Accuracy:  64.45%
Training: Epoch 4 || Loss:   2.068 || Accuracy:  66.63%
Training: Epoch 5 || Loss:   1.970 || Accuracy:  68.02%
Training: Epoch 6 || Loss:   1.827 || Accuracy:  69.07%
Training: Epoch 7 || Loss:   1.659 || Accuracy:  69.90%
Training: Epoch 8 || Loss:   1.527 || Accuracy:  70.62%
Training: Epoch 9 || Loss:   1.424 || Accuracy:  71.15%


0it [00:00, ?it/s]

...

Training: Epoch 480 || Loss:   0.878 || Accuracy:  76.32%
Training: Epoch 481 || Loss:   0.880 || Accuracy:  76.28%
Training: Epoch 482 || Loss:   0.881 || Accuracy:  76.28%
Training: Epoch 483 || Loss:   0.882 || Accuracy:  76.26%
Training: Epoch 484 || Loss:   0.882 || Accuracy:  76.24%
Training: Epoch 485 || Loss:   0.882 || Accuracy:  76.19%
Training: Epoch 486 || Loss:   0.884 || Accuracy:  76.21%
Training: Epoch 487 || Loss:   0.884 || Accuracy:  76.16%
Training: Epoch 488 || Loss:   0.883 || Accuracy:  76.16%
Training: Epoch 489 || Loss:   0.884 || Accuracy:  76.17%


0it [00:00, ?it/s]

Training: Epoch 490 || Loss:   0.886 || Accuracy:  76.15%
Training: Epoch 491 || Loss:   0.887 || Accuracy:  76.09%
Training: Epoch 492 || Loss:   0.887 || Accuracy:  76.11%
Training: Epoch 493 || Loss:   0.888 || Accuracy:  76.13%
Training: Epoch 494 || Loss:   0.888 || Accuracy:  76.06%
Training: Epoch 495 || Loss:   0.888 || Accuracy:  76.10%
Training: Epoch 496 || Loss:   0.888 || Accuracy:  76.05%
Training: Epoch 497 || Loss:   0.890 || Accuracy:  76.04%
Training: Epoch 498 || Loss:   0.891 || Accuracy:  76.02%
Training: Epoch 499 || Loss:   0.891 || Accuracy:  76.01%
Shuffling


0it [00:00, ?it/s]

Training: Epoch 500 || Loss:   0.883 || Accuracy:  76.02%


In [24]:
embeddings = torch.from_numpy(best_model.linear.weight.detach().cpu().numpy()).to(torch.float16)

In [25]:
zeroshot_weights = torch.from_numpy(np.array(embeddings).T).to(torch.float16)

def accuracy(output, target, topk=(1,)):
    pred = output.topk(max(topk), 1, True, True)[1].t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))
    return [
        float(correct[:k].reshape(-1).float().sum(0, keepdim=True).cpu().numpy())
        for k in topk
    ]

# lazy load
if clip_model == None:
    clip_model, clip_preprocess = clip.load(clip_model_name, device)

with torch.no_grad():
    top1, top5, n = 0.0, 0.0, 0.0
    for i, (images, target) in enumerate(tqdm(test_loader)):
        images = images.cuda()
        target = target.cuda()

        # predict

        image_features = clip_model.encode_image(images)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        logits = 100.0 * image_features.to(device) @ zeroshot_weights.to(device)

        # measure accuracy
        acc1, _ = accuracy(logits, target, topk=(1, 5))
        top1 += acc1
        n += images.size(0)

top1 = (top1 / n) * 100

print("acc:", top1)

  0%|          | 0/200 [00:00<?, ?it/s]

acc: 75.76


## Deterministic - (using sci-kit)

In [26]:
global clip_model, clip_preprocess
from sklearn.linear_model import LogisticRegression

len_classes = len(classes)

train_features, train_labels = get_clip_features(train_loader)
test_features, test_labels = get_clip_features(test_loader)

train_features = train_features / train_features.norm(dim=-1, keepdim=True)
test_features = test_features / test_features.norm(dim=-1, keepdim=True)

classifier = LogisticRegression(C=1, max_iter=1000, n_jobs=4,verbose=1)
classifier.fit(train_features.cpu().numpy(), train_labels.cpu().numpy())
predictions = classifier.predict(test_features.cpu().numpy())
accuracy = np.mean((test_labels.cpu().numpy() == predictions).astype(np.float)) * 100.0

print(accuracy)

  0%|          | 0/132 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


85.95848595848597


[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:   11.2s finished


In [8]:
clip_model, clip_preprocess = clip.load("ViT-B/32", device)
for param in clip_model.parameters

{}
